# Quick analysis

In [1]:
## Import libraries

%load_ext autoreload
%autoreload 2

import os
import pandas as pd
import numpy as np
from datetime import date, datetime
from IPython.display import display, Markdown
from utilities import redact_small_numbers


In [2]:
# import data

df = pd.read_feather("../output/input.feather")
#df = pd.read_csv("../output/input.csv")

# for binary fields, replace zeros with null so that `count` can
binary_fields = ['elective_short_stay', 'elective_or_op']
for f in binary_fields:
    df[f] = df[f].astype('int64').replace(0,np.nan)

maxdate = df["any_admission_date"].max()
display(Markdown(f"Latest admission date: {maxdate}"))
display(Markdown(f"Therapeutics: {df['outpatient_covid_therapeutic_name'].unique().tolist()}"))

Latest admission date: 2022-04-25

Therapeutics: ['paxlovid', 'molnupiravir', 'sotrovimab', nan, 'remdesivir', 'casirivimab and imdevimab']

In [3]:

fields = {
'Outpatient':
    ['outpatient_covid_therapeutic_date',
     'elective_or_op',
     'elective_short_stay', 
     'daycase_admission_date',
     'elective_x892_date', 
     'elective_x292_date', 
     'hospital_attendance_date'],
'Inpatient':      
    ['inpatient_covid_therapeutic_date',
     'any_admission_date', 
     'any_admission_x892_date',
     'any_admission_x292_date']
}

for x in fields:
    f = fields[x][0]
    display(Markdown(f"## {x}"))

    # filter to treatment dates within available SUS data range
    df1 = df.copy().loc[(df[f] <= maxdate)]

    if x=="Inpatient": # don't count admissions if discharge date was after treatment date
        display(Markdown(f"Note: for inpatients, recent spells may not yet have completed so some data may be missing"))
        for c in fields[x][1:]: # for each admission type
            # compare discharge date with treatment date and remove admission date if not in window
            df1.loc[df1[f] > df1[c.replace("admission", "discharge")], c] = np.nan
            
    # create df for mabs and separate df for Antivirals
    name_field = f'{x.lower()}_covid_therapeutic_name'
    df1[name_field] = df1[name_field].astype(str).fillna("")
    df_mab = df1.loc[df1[name_field].str.contains("mab")]
    df_non_mab = df1.loc[~df1[name_field].str.contains("mab")]
    treatments = {"MABs":df_mab,
                  "Antivirals":df_non_mab}

    # Breakdown by treatment type (MABs/Avs)
    for t in treatments:
        df_t = treatments[t]
        summary1 = pd.DataFrame(df_t[fields[x]].count()).rename(columns={0:"count"})
        summary1 = redact_small_numbers(summary1, n=5, rate_column=None)
        summary1["percent"] = (100*(summary1["count"]/summary1["count"][f])).fillna(0).round(1)
        summary1.index = summary1.index.str.replace("_date","")
        out_text = f"""Out of **{summary1["count"][0]}** patients treated in *{x}* settings with *{t}*, **{summary1["count"][1]} ({summary1["percent"][1]}%)** had *{summary1.index[1].replace("_"," ")}* in SUS"""

        display(Markdown(f"### {t}"), Markdown(out_text), summary1)


    # Breakdown by region and age group (MABs only, most inclusive hospital record field only)
    for col in ["region_covid_therapeutics","age_group"]:
        summary2 = pd.DataFrame(df_mab.groupby(col)\
                [fields[x][0:2]].count()).rename(columns={0:"count"})
        summary2 = redact_small_numbers(summary2, n=5, rate_column=None)
        summary2["percent"] = (100*summary2[fields[x][1]]/summary2[f]).fillna(0).round(1)
        summary2.columns = summary2.columns.str.replace("_date","")

        c_min = summary2["percent"].min()
        c_max = summary2["percent"].max()
        out_text = f"""The percent of MABs patients treated in *{x}* settings found in SUS varied by *{col}* from **{c_min}%** to **{c_max}%**"""
        display(Markdown("### Breakdown by region and age group (MABs only)"), Markdown(out_text), summary2)

## Outpatient

### MABs

Out of **280** patients treated in *Outpatient* settings with *MABs*, **140 (50.0%)** had *elective or op* in SUS

,count,percent
outpatient_covid_therapeutic,280,100.0
elective_or_op,140,50.0
elective_short_stay,205,73.2
daycase_admission,150,53.6
elective_x892,30,10.7
elective_x292,25,8.9
hospital_attendance,35,12.5


### Antivirals

...

The percent of MABs patients treated in *Inpatient* settings found in SUS varied by *region_covid_therapeutics* from **16.7%** to **50.0%**

,inpatient_covid_therapeutic,any_admission,percent
region_covid_therapeutics,,,
East,30,5,16.7
East Midlands,30,10,33.3
London,55,20,36.4
North East,30,10,33.3
North West,30,10,33.3
South East,40,15,37.5
South West,25,10,40.0
West Midlands,25,10,40.0
Yorkshire and The Humber,20,10,50.0


### Breakdown by region and age group (MABs only)

The percent of MABs patients treated in *Inpatient* settings found in SUS varied by *age_group* from **25.0%** to **50.0%**

,inpatient_covid_therapeutic,any_admission,percent
age_group,,,
12-24,35,15,42.9
25-34,20,5,25.0
35-44,55,15,27.3
45-54,45,15,33.3
55-64,35,10,28.6
65-74,35,15,42.9
75+,30,10,33.3
missing,30,15,50.0
